In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install textblob
!pip install vaderSentiment
!pip install transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from textblob import TextBlob

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import transformers

from transformers import pipeline,  AutoTokenizer

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_clean = pd.read_csv(r'/content/df_sentiment_vader_textblob.csv')

In [ ]:
df_clean.head(2)

,tweet_id,text,language,type,bookmark_count,favorite_count,retweet_count,reply_count,view_count,created_at,client,hashtags,urls,media_type,media_urls,user_id,name,n tweets,n followers,category,Factual Reporting,user,classification,emoji_count,text_clean,text_clean_sw,sentiment_textblob,sentiment_vader
0,'1861174452215529687',How do you express gratitude for one’s safety when the perils posed by the American police state grow more treacherous by the day?\nhttps://t.co/5O7Kc79dzn,en,Tweet,1.0,3.0,2.0,0.0,123.0,2024-11-25 22:25:43,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,https://activistpost.com/2024/11/divide-and-conquer-political-riptides-threaten-to-overwhelm-the-nation.html,NaN,NaN,ActivistPost,Activist Post,1000.0,41070.0,Pseudoscience,low,ActivistPost,fake news,0,How do you express gratitude for one’s safety when the perils posed by the American police state grow more treacherous by the day?,express gratitude one’s safety perils posed American police state grow treacherous day?,positive,positive
1,'1861172568218071498',We generally think of gold as a safe haven asset. But what about silver?\nhttps://t.co/fGLFe5ohf2,en,Tweet,1.0,0.0,0.0,0.0,125.0,2024-11-25 22:18:13,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,https://activistpost.com/2024/11/is-silver-a-safe-haven-asset.html,NaN,NaN,ActivistPost,Activist Post,1000.0,41070.0,Pseudoscience,low,ActivistPost,fake news,0,We generally think of gold as a safe haven asset. But what about silver?,generally think gold safe asset. silver?,positive,positive


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95734 entries, 0 to 95733
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            95734 non-null  object 
 1   text                95734 non-null  object 
 2   language            95684 non-null  object 
 3   type                95734 non-null  object 
 4   bookmark_count      95684 non-null  float64
 5   favorite_count      95684 non-null  float64
 6   retweet_count       95684 non-null  float64
 7   reply_count         95684 non-null  float64
 8   view_count          88284 non-null  float64
 9   created_at          95734 non-null  object 
 10  client              95684 non-null  object 
 11  hashtags            17439 non-null  object 
 12  urls                57777 non-null  object 
 13  media_type          40391 non-null  object 
 14  media_urls          40391 non-null  object 
 15  user_id             95734 non-null  object 
 16  name

In [ ]:
model_name = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_pipeline = pipeline('sentiment-analysis', model=model_name, tokenizer=tokenizer)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
text = ["Scientists uncovered new chemical compound lurking U.S. drinking water, potentially exposing one five Americans unidentified health risk."]
sentiment_pipeline(text)

[{'label': 'negative', 'score': 0.595809280872345}]

In [ ]:
# Counts the number of tweets containing more than 512 tokens
def count_long_tweets(df, column_name='text_clean', max_length=510):
    long_tweet_count = 0
    for text in df[column_name]:
        if isinstance(text, str):
            tokens = tokenizer.tokenize(text)
            if len(tokens) > max_length:
                long_tweet_count += 1
    return long_tweet_count

long_tweet_count = count_long_tweets(df_clean, column_name='text_clean')

print(f"Number of tweets with more than 512 tokens: {long_tweet_count}")

Number of tweets with more than 512 tokens: 101


In [ ]:
def identify_long_tweets(df, column_name='text_clean', max_length=510):
    long_tweet_indices = []
    for index, text in enumerate(df[column_name]):
        if isinstance(text, str):  # Verificar se o texto é uma string
            tokens = tokenizer.tokenize(text)
            if len(tokens) > max_length:
                long_tweet_indices.append(index)
    return long_tweet_indices

long_tweet_indices = identify_long_tweets(df_clean, column_name='text_clean')

# Remove rows containing tweets with more than 512 tokens
df_tweets = df_clean.drop(long_tweet_indices)

In [ ]:
df_tweets = df_tweets.dropna(subset=['text_clean'])


In [ ]:
def analyze_sentiment(text):
    sentiment_result = sentiment_pipeline(text)[0]
    return pd.Series([sentiment_result['label'], sentiment_result['score']])

In [ ]:
df_tweets[['sentiment_analysis', 'sentiment_score']] = df_tweets['text_clean'].apply(lambda x: pd.Series(analyze_sentiment(x)))


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df_tweets.head(5)

,tweet_id,text,language,type,bookmark_count,favorite_count,retweet_count,reply_count,view_count,created_at,client,hashtags,urls,media_type,media_urls,user_id,name,n tweets,n followers,category,Factual Reporting,user,classification,emoji_count,text_clean,text_clean_sw,sentiment_textblob,sentiment_vader,sentiment_analysis,sentiment_score
0,'1861174452215529687',How do you express gratitude for one’s safety when the perils posed by the American police state grow more treacherous by the day?\nhttps://t.co/5O7Kc79dzn,en,Tweet,1.0,3.0,2.0,0.0,123.0,2024-11-25 22:25:43,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,https://activistpost.com/2024/11/divide-and-conquer-political-riptides-threaten-to-overwhelm-the-nation.html,NaN,NaN,ActivistPost,Activist Post,1000.0,41070.0,Pseudoscience,low,ActivistPost,fake news,0,How do you express gratitude for one’s safety when the perils posed by the American police state grow more treacherous by the day?,express gratitude one’s safety perils posed American police state grow treacherous day?,positive,positive,negative,0.826268
1,'1861172568218071498',We generally think of gold as a safe haven asset. But what about silver?\nhttps://t.co/fGLFe5ohf2,en,Tweet,1.0,0.0,0.0,0.0,125.0,2024-11-25 22:18:13,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,https://activistpost.com/2024/11/is-silver-a-safe-haven-asset.html,NaN,NaN,ActivistPost,Activist Post,1000.0,41070.0,Pseudoscience,low,ActivistPost,fake news,0,We generally think of gold as a safe haven asset. But what about silver?,generally think gold safe asset. silver?,positive,positive,neutral,0.880189
2,'1861093943267520824',"We Can’t Get Hurricane Victims In North Carolina The Help They Need, But We Have $174 Billion For The War In Ukraine? - https://t.co/RZBJmzPhRf",en,Tweet,0.0,1.0,1.0,0.0,175.0,2024-11-25 17:05:48,"<a href=""http://www.mailchimp.com"" rel=""nofollow"">Mailchimp</a>",NaN,http://eepurl.com/i4T0JI,NaN,NaN,ActivistPost,Activist Post,1000.0,41070.0,Pseudoscience,low,ActivistPost,fake news,0,"We Can’t Get Hurricane Victims In North Carolina The Help They Need, But We Have $174 Billion For The War In Ukraine?","Can’t Get Hurricane Victims North Carolina Help Need, $174 Billion War Ukraine?",neutral,negative,negative,0.855999
3,'1861069708536991847',"Scientists have uncovered a new chemical compound lurking in U.S. drinking water, potentially exposing one in five Americans to an unidentified health risk.\nhttps://t.co/kbL1Hm2kBX",en,Tweet,1.0,2.0,3.0,0.0,176.0,2024-11-25 15:29:30,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,https://naturalblaze.com/2024/11/researchers-find-unknown-chemical-in-drinking-water-posing-potential-human-health-concern.html,NaN,NaN,ActivistPost,Activist Post,1000.0,41070.0,Pseudoscience,low,ActivistPost,fake news,0,"Scientists have uncovered a new chemical compound lurking in U.S. drinking water, potentially exposing one in five Americans to an unidentified health risk.","Scientists uncovered new chemical compound lurking U.S. drinking water, potentially exposing one five Americans unidentified health risk.",positive,negative,negative,0.669265
4,'1860817334283202825',"With winter looming over the Appalachian region, at least a thousand people—some of them children—are still living in temporary housing, such as tents.\nhttps://t.co/dniKY5IgII",en,Tweet,0.0,2.0,1.0,0.0,264.0,2024-11-24 22:46:39,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,https://activistpost.com/2024/11/major-cold-blast-to-hit-helene-ravaged-nc-with-families-living-in-tents-cars-amid-fema-blunder.html,NaN,NaN,ActivistPost,Activist Post,1000.0,41070.0,Pseudoscience,low,ActivistPost,fake news,0,"With winter looming over the Appalachian region, at least a thousand people some of them children are still living in temporary housing, such as tents.","winter looming Appalachian region, least thousand people childre

In [ ]:
df_tweets.to_csv('df_all_sentiment_tensor.csv', index=False)